In [2]:
import numpy as np
import pandas as pd
import os
from urllib.parse import quote, unquote
import pickle

In [20]:
path = '/home/ctang/ttmp/MMCSR/GiantMIDI'
file_path = path + '/full_music_pieces_youtube_similarity_pianosoloprob_split.csv'
GiantMidi = pd.read_csv(file_path)

In [21]:
GiantMidi = GiantMidi[['surname', 'firstname', 'music', 'audio_name']]

In [22]:
GiantMidi['midi_path'] = ''
GiantMidi['midi_path'] = '/home/ctang/ttmp/MMCSR/GiantMIDI/midis/' + GiantMidi['audio_name'] + '.mid'
GiantMidi.drop(columns=['audio_name'], inplace=True)
GiantMidi.head()

,surname,firstname,music,midi_path
0,A.,Jag,Je t'aime Juliette,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/A., Jag..."
1,Aadler,C. A.,Floating Islands,NaN
2,Aagesen,Truid,Cantiones trium vocum,NaN
3,Aaron,Michael,Piano Course,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/Aaron, ..."
4,Aarons,Alfred E.,Brother Bill,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/Aarons,..."


In [23]:
with open('/home/ctang/ttmp/MMCSR/pretraining_metadata/all_metadata.pkl', 'rb') as f:
    all_metadata = pickle.load(f)

In [24]:
GiantMidi['metadata'] = False

In [25]:
for row in GiantMidi.itertuples():
    surname = row.surname
    firstname = row.firstname
    title = row.music
    midi_path = row.midi_path
    composer = unquote(surname + ', ' + firstname)
    if ',' in title and any(char.isdigit() for char in title):
        title = ",".join(title.split(",")[:-1])
    title = unquote(title)
    if composer in all_metadata and title in all_metadata[composer]:
        GiantMidi.at[row.Index, 'metadata'] = True

In [26]:
GiantMidi = GiantMidi[GiantMidi['metadata'] == True]

In [27]:
copy = GiantMidi.copy()

# drop duplicates
copy.drop_duplicates(subset=['surname', 'firstname', 'music'], inplace=True)

In [29]:
# drop rows where midi_path is nan
copy.dropna(subset=['midi_path'], inplace=True)
copy.reset_index(drop=True, inplace=True)

In [31]:
# remove my personal information from midi_path
copy['midi_path'] = copy['midi_path'].apply(lambda x: x.replace('/home/ctang/ttmp/MMCSR/', '../'))

In [33]:
# change music to title
copy.rename(columns={'music': 'title'}, inplace=True)

# add composer name column
copy['composer'] = copy['surname'] + ', ' + copy['firstname']

In [34]:
# drop surname, firstname, and metadata columns
copy.drop(columns=['surname', 'firstname', 'metadata'], inplace=True)

In [37]:
# save to csv
copy.to_csv('/home/ctang/ttmp/MMCSR/GiantMIDI/GiantMidi.csv', index=False)